In [1]:
from hllset_swarm.trajectory import SwarmProgram
from hllset_swarm.io.env import Environment

__all__ = ["SwarmProgram", "Environment"]

Loading HLLSet kernel from: None


In [2]:
from hllset_swarm.hllset_wrapper import HllSet

h = HllSet(P=10)